In [239]:
from selenium import webdriver
from time import sleep 
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import pandas as pd
import csv
from datetime import date
import re
import numpy as np

In [117]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

# Extração

## Definindo Super Classe

In [118]:
class CarregarPagina:
    """
    O CarregarPagina é uma Super classe utilitária que facilita o acesso a páginas da web   
    """
    def __init__(self, url, driver=None):
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")
        self.driver = webdriver.Chrome(chrome_options=options) if not driver else driver
        self.url = url
        self.driver.get(self.url)


## Definindo Classes
Nessa sessão definimos as classes específicas para cada uma das páginas das universidades das quais queremos extrair informações. Essas classes extraem e fazem o pré tratamento dos dados que julgamos importantes para nosso resultado final.

### Extrator UEPG

In [119]:
class ExtratorUEPG(CarregarPagina):
    """
    O ExtratorUEPG é uma classe utilitária que facilita a extração das informações sobre o vestibular de verão atual da Univesidade Estadual de Ponta Grossa (UEPG)   
    """
    def __init__(self,  url = "https://cps.uepg.br/inicio/"):
        """
        Inicializador

        url = endereço da página desejada
        lst_informações = armazena as iformações coletadas
        qntd_linhas = define o número de linhas a serem percorridas pelo nosso extrator
        """
        super().__init__(url)
        self.lst_informações = []
        self.qntd_linhas = 9

   
    def pagina_vestibular(self):
        """
        Seleciona o último vestibular para visualizar mais detalhes sobre
        """
        # Clica no ùltimo vestibular postado
        vestibular = self.driver.find_element_by_xpath("""//*[@id="content-1column"]/section/div/article/table/tbody/tr[3]/td[1]/a[1]""")
        vestibular.click()

    def informações_vestibular(self):
        """
        Extrai as informações do vestibular
        """
        # Grava o nome do vestibular
        vestibular_atual = self.driver.find_element_by_xpath("""//*[@id="content-1column"]/article/table/tbody/tr[2]/td/p[1]/span""")
        self.lst_informações.append(vestibular_atual.text)
        
        # Extrai as informações como data de abertura e encerramento de inscrições, data de liberação dos resultados, taxa de inscrição etc.
        for i in range (1, self.qntd_linhas):
            xpath_linhas = f'//*[@id="content-1column"]/article/table/tbody/tr[3]/td/table/tbody/tr/td[2]/table/tbody/tr[{i}]'
            informações = self.driver.find_element_by_xpath(xpath_linhas)
            self.lst_informações.append(informações.text)

    def atribuir_datas(self):
        '''
        correlaciona e formata a lista de datas e de títulos
        '''
        for linha in self.lst_informações:
            if 'Período de inscrição' in linha:
                texto = linha.split(':')
                texto = str(texto[1])
                data = texto.split('a')
                self.inicio_das_insc = data[0]
                self.termino_das_insc = data[1]
            elif 'Aplicação das provas' in linha:
                texto = linha.split(':')
                texto = str(texto[1])
                self.primeira_fase = texto

    def extrair_informacoes(self):
        '''
        Reúne tdos os métodos anteriores em apenas um
        '''
        print('Carregando página UEPG...')
        self.pagina_vestibular()
        print('Extraindo Informações...')
        self.informações_vestibular()
        print('Armazenando dados...')
        self.atribuir_datas()
        print('concluído!')


### Extrator Nucvest

In [120]:

class ExtratorNUCVEST(CarregarPagina):
    """
    O ExtratorNUCVEST é uma classe utilitária que facilita a extração das informações sobre o atual vestibular de verão realizado pela NUCVEST  
    """
    def __init__(self, url = 'https://www.nucvest.com.br/'):
        """
        Inicializador

        url = endereço da página desejada
        elem_pag = ?????????????
        inscricao = armazena informações sobre a inscrição no vestibular
        """
        super().__init__(url)
        self.elem_pag = 20
        self.inscricao = []
        
    def aceitarcookies(self):
        ''' 
        Aceita os cookies da página
        '''
        self.btn_cookie = self.driver.find_element_by_class_name("""fundasp-lgpd-button-accept""")
        self.btn_cookie.click()
    
    def cronograma(self):
        '''
        abre a página que contem os dados referente ao cronograma 
        '''
        btn = self.driver.find_element_by_class_name("""calendar""")
        btn.click()
        
    def extrair_info_páginas(self):
        ''' 
        coleta as informações uteis para nosso framework
        '''
        informacoes = []
        for i in range(1, self.elem_pag+1):
            data = self.driver.find_element_by_xpath(f"""//*[@id="collapseTwo"]/div/ul/li[{i}]/div[1]""")
            data = data.text
            try:
                conteudo = self.driver.find_element_by_xpath(f"""//*[@id="collapseTwo"]/div/ul/li[{i}]/div[2]""")
                conteudo = conteudo.text
            except:
                conteudo = ''
            dados = {'Data':data,
                    'Conteudo':conteudo}
            informacoes.append(dados)
        self.inscricao = informacoes

    def formatar_dados(self):
        ''' 
        Faz uma formatação incial dos dados para que possam ser armasenado em nosso banco de dados
        '''
        for linha in self.inscricao:
            if 'Período de Inscrições' in str(linha['Conteudo']):
                data = str( linha['Data']).split('a')
                print(data[0])
                self.inicio_das_insc = data[0]
                self.termino_das_insc = data[1]
            elif 'Aplicação da prova' in str(linha['Conteudo']):
                self.primeira_fase = str(linha['Data']).split('a')

    def extrair_informacoes(self):
        ''' 
        Junta todos os métodos anteriores em apenas um
        '''
        print('Carregando página Nucvest...')
        self.aceitarcookies()
        self.cronograma()
        print('Extraindo Informações...')
        self.extrair_info_páginas()
        print('Armazenando dados...')
        self.formatar_dados()
        print('concluído!')

### Unifesp

In [121]:
class ExtratorUnifesp(CarregarPagina):
    """
    O ExtratorUnifesp é uma classe utilitária que facilita a extração das informações sobre o atual vestibular de verão da Universidade Federal do Estado de São Paulo  

    inscricao = armazena os dados referentes a inscrição 
    """
    def __init__(self, url = 'https://ingresso.unifesp.br/informacoes-fixas/informacoes-fixas-misto/cronograma-sistema-misto'):
        super().__init__(url)
        self.inscricao = []
        
    def extrair_info_páginas(self):
        '''
        Coleta da página os dados relevantes para o nosso projeto
        '''
        dados_lst = []
        for k in range(1, 12):
            data = self.driver.find_element_by_xpath(f"""//*[@id='g-main']/div/div/div/div/div/div/div/div[2]/table/tbody/tr[{k}]/td[1]""")
            data = data.text
            try:
                conteudo = self.driver.find_element_by_xpath(f"""//*[@id='g-main']/div/div/div/div/div/div/div/div[2]/table/tbody/tr[{k}]/td[2]""")
                conteudo = conteudo.text
            except:
                conteudo = ''
            dados = {'Data':data,
                    'Conteudo':conteudo}
            dados_lst.append(dados)
        self.inscricao = dados_lst
        
    def formatar_dados(self):
        '''
        Realiza a pré-formatação dos dados para serem adicionados ao nosso banco de dados 
        '''
        for linha in self.inscricao:
            if  'Período para pedido de isenção de taxa de inscrição do vestibular' in str(linha['Conteudo']):
                data = str( linha['Data']).split('às')
                self.inicio_isencao = data[0]
                self.termino_isencao = data[1]
            elif 'Período de inscrição para as provas complementares' in str(linha['Conteudo']):
                data = str( linha['Data']).split('às')
                self.inicio_das_insc = data[0]
                self.termino_das_insc = data[1]
            elif 'Provas Complementares:\n' in str(linha['Conteudo']):
                data = str(linha['Data']).split('\n')
                self.primeira_fase = data[0]
                self.primeira_fase_dia2 = data[1]
    
    def extrair_informacoes(self):
        ''' 
         Junta todos os métodos anteriores em apenas um
        '''
        print('Extraindo informações Unifesp...')
        self.extrair_info_páginas()
        print('Formatando dados...')
        self.formatar_dados()
        print('Concluído!')
            

### Acafe

In [122]:
class ExtratorAcafe(CarregarPagina):
    def __init__(self, url = 'https://acafe.org.br/site/', periodo = 'Verão 2022'):
        '''
        O ExtratorAcafe é uma classe utilitária que facilita a extração das informações sobre o atual vestibular de verão fornecido pela Acafe  

        periodo = Qual edição da prova queremos (ex:'Verão 2022', 'Inverno 2022', ...)
        '''
        super().__init__(url)
        self.periodo =  periodo


    def abrir_calendario(self):
        '''  
        Abre o banner html que possui o calendário do vestibular
        '''
        #fechar banner
        try:
            sleep(15)
            banner = self.driver.find_element_by_xpath(f'''/html/body/div[3]/div/div/div[3]/button''')
            banner.click()
            print('banner fechado.')
        except:
            pass
        #Aceita os cookies e abre o calendário 
        aceitar_cookies = self.driver.find_element_by_class_name(f'''btn-cookies''')
        print('cookies aceitos.')
        abrir_calendario = self.driver.find_element_by_xpath(f'''/html/body/div[5]/div/div[2]''')
        print('Calendário aberto.')
        sleep(5)
        aceitar_cookies.click()
        sleep(3)
        abrir_calendario.click()    

    def extrair_conteudo(self):
        '''  
        Extrai da página o conteúdo que julgamos relevante para nosso trabalho
        '''
        #extrai o conteúdo do calendário (os acontecimentos que queremos para o nosso banco de dados e suas respectivas datas)
        self.abrir_calendario()
        self.titulos_element = self.driver.find_elements_by_tag_name(f'''th''')
        self.linhas_element = self.driver.find_elements_by_tag_name(f'''td''')

        #organizando conteúdo em listas
        linhas = []
        for linha in self.linhas_element:
            linhas.append(linha.text)
        self.linhas = linhas
        titulos = []
        for titulo in self.titulos_element:
            if titulo.text != '':
                titulos.append(titulo.text)
        self.titulos = titulos

    def atribuir_datas(self):
        '''
        pré-formata e atribui as datas a cada acontecimento importante refernte às provas, para adicionarmos a nossa base de dados
        '''
        #correlacionando e formatando a lista de datas e de títulos(acontecimento)
        self.indice_do_perido = int(self.titulos.index(self.periodo))
        for i in range(0,len(self.linhas), len(self.titulos)):
            if self.linhas[i] == 'Aplicação da prova':
                self.primeira_fase = self.linhas[(i+self.indice_do_perido)]
            elif self.linhas[i] == 'Inscrição':
                datas = (self.linhas[(i+self.indice_do_perido)]).split('a')
                self.inicio_das_insc = datas[0]
                self.termino_das_insc = datas[1]  
            elif self.linhas[i]   == 'Pagamento':
                datas = (self.linhas[(i+self.indice_do_perido)]).split('Até')
                self.inicio_pagamento = datas[0]
                self.termino_pagamento = datas[1]

    def extrair_informacoes(self):
        '''
        método que ao ser chamado, executa todas os anteriores.
        '''
        self.extrair_conteudo()
        self.atribuir_datas()


## Realizando Extração

### UEPG

In [123]:
uepg = ExtratorUEPG()
uepg.extrair_informacoes()

C:\Users\marin\AppData\Local\Temp/ipykernel_10768/1654213129.py:8: DeprecationWarning: use options instead of chrome_options
  self.driver = webdriver.Chrome(chrome_options=options) if not driver else driver


Carregando página UEPG...
Extraindo Informações...
Armazenando dados...
concluído!


### Nucvest

In [124]:
nucvest = ExtratorNUCVEST()
nucvest.extrair_informacoes()

C:\Users\marin\AppData\Local\Temp/ipykernel_10768/1654213129.py:8: DeprecationWarning: use options instead of chrome_options
  self.driver = webdriver.Chrome(chrome_options=options) if not driver else driver


Carregando página Nucvest...
Extraindo Informações...
Armazenando dados...
11/10 
concluído!


### Unifesp

In [125]:
unifesp = ExtratorUnifesp()
unifesp.extrair_informacoes()

C:\Users\marin\AppData\Local\Temp/ipykernel_10768/1654213129.py:8: DeprecationWarning: use options instead of chrome_options
  self.driver = webdriver.Chrome(chrome_options=options) if not driver else driver


Extraindo informações Unifesp...
Formatando dados...
Concluído!


### Acafe

In [126]:
acafe = ExtratorAcafe()
acafe.extrair_informacoes()

C:\Users\marin\AppData\Local\Temp/ipykernel_10768/1654213129.py:8: DeprecationWarning: use options instead of chrome_options
  self.driver = webdriver.Chrome(chrome_options=options) if not driver else driver


banner fechado.
cookies aceitos.
Calendário aberto.


# Incluindo no dataset

In [155]:
class ObterDados:
    ''' 
    Adicona os dados coletados à tabela principla
    '''
    def __init__(self, file = 'ranking_universidades.csv'):
        '''   
        abre a tabela


        ranking = dataset com a tabela principal
        '''
        self.ranking = pd.read_csv(file, dtype = str, encoding = 'latin-1', sep = ';')
        self.ranking.fillna('', inplace = True)
    
    def adicionar_a_tabela(self):
        ''' 
        adicona os dados coletados ao dataset
        '''
        for index, row in self.ranking.iterrows():
            if row['Extrator'] == 'ExtratorAcafe':
                row['link site'] = acafe.url
                row['Inicio incrição'] = acafe.inicio_das_insc
                row['Fim inscrição'] = acafe.termino_das_insc
                row['Primiera fase'] = acafe.primeira_fase
            elif row['Extrator'] == 'ExtratorUepg':
                row['link site'] = uepg.url
                row['Inicio incrição'] = uepg.inicio_das_insc
                row['Fim inscrição'] = uepg.termino_das_insc
                row['Primiera fase'] = uepg.primeira_fase
            elif row['Extrator'] == 'ExtratorUnifesp':
                row['link site'] = unifesp.url
                row['Inicio incrição'] = unifesp.inicio_das_insc
                row['Fim inscrição'] = unifesp.termino_das_insc
                row['Primiera fase'] = unifesp.primeira_fase
                row['Primeira fase (segundo dia)'] = unifesp.primeira_fase_dia2
            elif row['Extrator'] == 'ExtratorNucvest':
                row['link site'] = nucvest.url
                row['Inicio incrição'] = nucvest.inicio_das_insc
                row['Fim inscrição'] = nucvest.termino_das_insc
                row['Primiera fase'] = nucvest.primeira_fase

    def salvar(self):
        '''  
        salva o dataset em csv
        '''
        self.ranking.to_csv('rankingteste.csv', sep = ',', encoding = 'utf-8', index = False, quoting = csv.QUOTE_ALL)

In [156]:
ranking = ObterDados()
ranking.adicionar_a_tabela()

# Tratamento

In [157]:
class Tratando():
    def __init__(self, df, colunas_data = ['Inicio incrição', 'Fim inscrição', 'Início isenção ', 'Fim isenção',
    'Primiera fase', 'Primeira fase (segundo dia)', 'Segunda fase',
    'Segunda fase (segundo dia)', 'Resultado']):
        self.df = df
        self.colunas_data = colunas_data
        self.coluna_metadados = 'metadados'

    def info_adicional(self, texto):
        info_adicional = [ 
        'ENEM'
        'EAD'
        'SISU'
        'Cancelado'
        ]
        texto_final = ''
        for info in info_adicional:
            if info in texto:
                texto_final += f'''{info}\n''' 
        return f'''{self.coluna}: {texto_final}''' if texto_final != '' else ''

    def encontrar_datas(self, texto):
        r = re.compile(r'\d{2}/\d{2}/\d{4}')
        s =  re.compile(r'(\d{2}/\d{2}[^/0-9])')
        texto = str(texto)
        r = str(r.findall(texto))[2:-2]
        s = str(s.findall(texto))[2:-3]
        if r != '':
            self.dia = r[:2]
            self.mes = r[3:5]
            self.ano = r[6:]
            self.data = f'''{self.ano}-{self.mes}-{self.dia}'''
        elif s != '':
            self.dia = s[:2]
            self.mes = s[3:5]
            self.data = f'''-{self.mes}-{self.dia}'''
        else:
            self.data = ''
            self.metadados = self.info_adicional(texto)
    
    def adicionar_ano(self, texto):
        r = re.compile(r'\d{2}/\d{2}/\d{4}')
        r = str(r.findall(texto))[2:-2]
        if r != '':
            self.proximo_dia = r[:2]
            self.proximo_mes = r[3:5]
            self.proximo_ano = r[6:]
        else:
            self.proximo_dia = ''
            self.proximo_mes = ''
            self.proximo_ano = ''

    def tratar_datas(self):
        for index, row in self.df.iterrows():
            for i in range(len(self.colunas_data)):
                self.coluna = self.colunas_data[i] 
                try:
                    self.proxima_coluna = self.colunas_data[i+1] #caso a data não tenha o ano, pegamos da próxima data (ex: incrições de 20/10 a 20/11/2021)
                    self.adicionar_ano(row[self.proxima_coluna])
                except:
                    self.proxima_coluna = ''
                self.metadados = ''
                self.encontrar_datas(row[self.coluna])
                if self.data != '':
                    if str(self.data)[0] == '-':
                        self.data = str(self.proximo_ano) + str(self.data)
                self.df.loc[index, self.coluna_metadados] += self.metadados
                self.df.loc[index, self.coluna] = self.data

    def salvar(self):
        '''  
        salva o dataset em csv
        '''
        self.df = self.df.drop(columns=['Extrator'])
        self.df.to_csv('ranking_tratado.csv', sep = ',', encoding = 'utf-8', index = False, quoting = csv.QUOTE_ALL)
            

In [158]:
df_completo = ranking.ranking
df_tratado = Tratando(df_completo)
df_tratado.tratar_datas()

# Visualização

In [280]:
class Ordenar():
    def __init__(self, df, tirar_sem_datas = False):
        '''
        Ordena a tabela de a cordo com a preferência do usuário

        df = dataframe utilizado
        tirar_sem_datas = tira da visualização as universidades que não possuem datas para nenhum dos processos.
        '''
        self.df = df
        colunas_int = [
            'Ranking', 'Posição em Pesquisa', 'Posição em Ensino', 'Posição em Mercado', 'Posição em Inovação', 'Posição em Internacionalização'
            ]
        self.colunas_data = [
            'Inicio incrição', 'Fim inscrição', 'Início isenção ', 'Fim isenção', 'Primiera fase', 
            'Primeira fase (segundo dia)', 'Segunda fase', 'Segunda fase (segundo dia)',	'Resultado'
            ]
        self.colunas_colocacao = [ 
            'Posição em Ensino', 'Posição em Pesquisa',  'Posição em Mercado',  'Posição em Inovação', 
            'Posição em Internacionalização', 'Nota em Internacionalização', 'Nota'
        ]

        if tirar_sem_datas == 'True':
            self.df = self.df.replace('', np.nan)
            self.df = self.df.dropna(how = 'all', subset = [self.colunas_data])
            self.df = self.df.fillna('')
            
        self.estados = df.Estado.unique()
        self.df = self.df.rename(columns={'metadados':'Mais Informações'})
        self.df[colunas_int] = self.df[colunas_int].astype(int)
        self.df['Nota'] = self.df['Nota'].astype(float)
        
    def help(self):
        print(f'''Lista de comandos:
    .ruf(\"parametro\") -> Ordena as universidade de acordo com o a RUF 2019, da mais bem colocada para a menos
    \tparâmetros:
    \t\t'Posição em Ensino', 'Posição em Pesquisa',  'Posição em Mercado',  'Posição em Inovação', 
    \t\t'Posição em Internacionalização', 'Nota em Internacionalização', 'Nota'.
    .publica() -> Exibe apenas as universidades públicas (Federais, Estaduais ou Municipais).
    .privada() -> Exibe apenas as universidades privadas.
    .estado(\"estado\") -> Exibe as faculdades do estado passado como parâmetro.
    .processo(\"etapa\", \"periodo\") -> Ordena a tabela da data mais antiga para a mais recente do processo escolhido.
    \tparâmetros: 
    \t\tetapa = \"ins\" (inscrição), \"ise\" (Isenção), \"pri\" (Primeira fase), \"seg\" (Segunda Fase), \"res\" (Resultado)
    \t\tperido = \"i\" (início), \"f\" (fim), \"p\" (primeiro dia), \"s\" (segundo dia)''')

    def ruf(self, parametro : str):
        if parametro in self.colunas_colocacao:
            df_ = self.df 
            df_ = df_.sort_values(by = parametro)
            return display(df_)
        else:
            print(f'''\"{parametro}\" não é válido, confia a lista de parâmetros válidos:\n{self.colunas_colocacao}''')

    def publica(self):
        df_ = self.df
        df_ = df_.loc[df_['Pública ou Privada']!='Privada']
        return display(df_)

    def privada(self):
        df_ = self.df
        df_ = df_.loc[df_['Pública ou Privada']=='Privada']
        return display(df_)

    def estado(self, sigla_estado : str):
        if sigla_estado in self.estados:
            df_ = self.df
            df_ = df_.loc[df_['Estado']== sigla_estado]
            return display(df_)
        else:
            return print(f'''\"{sigla_estado}\" não é um estado válido. Confira a lista de estados válidos: \n{self.estados}''')
    
    def ensino(self):
        df_ = self.df 
        df_ = df_.sort_values(by = 'Posição em Ensino')
        return display(df_)
    
    def coincide(self, etapa : str):
        if  etapa in self.colunas_data:
            df_ = self.df 
            df_ = pd.concat(linha for _, linha in df_.groupby(etapa) if len(linha) > 1)
            df_ = df_.loc[df_[etapa]!= '']
            print(f'''Datas coincidentes em \"{etapa}\"''')
            return display(df_)
        else:
             print(f'''\"{etapa}\" não é válido, confia a lista de parâmetros válidos:\n{self.colunas_data}''')

    def processo(self, etapa, periodo):
        df_ = self.df
        df_[self.colunas_data] =  df_[self.colunas_data].apply(pd.to_datetime, format='%Y-%m-%d')
        if etapa == 'ins':
            if periodo == 'i': 
                df_ = df_.sort_values(by = 'Inicio incrição')
                return df_.replace({pd.NaT: ""})
            elif periodo == 'f':
                df_ = df_.sort_values(by = 'Fim inscrição')
                return df_.replace({pd.NaT: ""})
            else:
                print(f'''\"{periodo}\" é um período inválido para Inscrição.
Use \".help()\" para verificar a lista de parâmetros possíveis para este comando.''')
        elif etapa == 'ise':
            if periodo == 'i': 
                df_ = df_.sort_values(by = 'Início isenção ')
                return df_.replace({pd.NaT: ""})
            elif periodo == 'f':
                df_ = df_.sort_values(by = 'Fim isenção')
                return df_.replace({pd.NaT: ""})
            else:
                print(f'''\"{periodo}\" é um período inválido para Isenção de Taxa.
Use \".help()\" para verificar a lista de parâmetros possíveis para este comando.''')
        elif etapa == 'pri':
            if periodo == 'p':
                df_ = df_.sort_values(by = 'Primiera fase')
                return df_.replace({pd.NaT: ""})
            elif periodo == 's':
                df_ = df_.sort_values(by = 'Primeira fase (segundo dia)')
                return df_.replace({pd.NaT: ""})
            else:
                print(f'''\"{periodo}\" é um período inválido para Primeira fase.
Use \".help()\" para verificar a lista de parâmetros possíveis para este comando.''')
        elif etapa == 'seg':
            if periodo == 'p':
                df_ = df_.sort_values(by = 'Segunda fase')
                return df_.replace({pd.NaT: ""})
            elif periodo == 's':
                df_ = df_.sort_values(by = 'Segunda fase (segundo dia)')
                return df_.replace({pd.NaT: ""})
            else:
                print(f'''\"{periodo}\" é um período inválido para Segunda fase.
Use \".help()\" para verificar a lista de parâmetros possíveis para este comando.''')
        elif etapa == 'res':
            df_ = df_.sort_values(by = 'Resultado')
            return df_.replace({pd.NaT: ""})
        else:
            print(f'''\"{etapa}\" é uma etapa inválida.\nUse \".help()\" para verificar a lista de parâmetros possíveis para este comando.''')

In [282]:
tabela = Ordenar(df_tratado.df)
tabela.help()

Lista de comandos:
    .ruf("parametro") -> Ordena as universidade de acordo com o a RUF 2019, da mais bem colocada para a menos
    	parâmetros:
    		'Posição em Ensino', 'Posição em Pesquisa',  'Posição em Mercado',  'Posição em Inovação', 
    		'Posição em Internacionalização', 'Nota em Internacionalização', 'Nota'.
    .publica() -> Exibe apenas as universidades públicas (Federais, Estaduais ou Municipais).
    .privada() -> Exibe apenas as universidades privadas.
    .estado("estado") -> Exibe as faculdades do estado passado como parâmetro.
    .processo("etapa", "periodo") -> Ordena a tabela da data mais antiga para a mais recente do processo escolhido.
    	parâmetros: 
    		etapa = "ins" (inscrição), "ise" (Isenção), "pri" (Primeira fase), "seg" (Segunda Fase), "res" (Resultado)
    		perido = "i" (início), "f" (fim), "p" (primeiro dia), "s" (segundo dia)


In [148]:
df_tratado.df.columns

Index(['Ranking', 'Instituição', 'Pública ou Privada', 'Estado',
       'Inicio incrição', 'Fim inscrição', 'Início isenção ', 'Fim isenção',
       'Primiera fase', 'Primeira fase (segundo dia)', 'Segunda fase',
       'Segunda fase (segundo dia)', 'Resultado', 'link site', 'metadados',
       'Posição em Ensino', 'Nota em Ensino', 'Posição em Pesquisa',
       'Nota em Pesquisa', 'Posição em Mercado', 'Nota em Mercado',
       'Posição em Inovação', 'Nota em Inovação',
       'Posição em Internacionalização', 'Nota em Internacionalização', 'Nota',
       'Extrator'],
      dtype='object')